In [ ]:
# zepyra
# LLama
# Komodo

# common imports

import sys
assert sys.version_info >= (3, 5)


import sklearn
assert sklearn.__version__ >= "0.20"

import datasets

import numpy as np
import pandas as pd


import os
import csv
import json


import re
from tqdm import tqdm

np.random.seed(42)

In [ ]:
def load_dataset(path = ''): 
    with open(path, 'r',encoding="utf8") as dataset:
        dataset =  [json.loads(line) for line in dataset.readlines()]
    return dataset

In [ ]:
detect_rewrite_dataset_path = 'E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_for_training_17042024\\detect_rewrite.jsonl'
generate_dataset_path = 'E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_for_training_17042024\\generate.jsonl'
detect_rewrite_dataset = load_dataset(detect_rewrite_dataset_path)
generate_dataset = load_dataset(generate_dataset_path)

In [ ]:
test_ids = [23140,4940,23941,21720,20952,28919,20643,16568,69,230,461,20645,21467,19976,1582,1907,1684,2012,2215,2313,2418,3198,3310,3599,3957,4229,4565,5476]
print(len(test_ids))

detect_rewrite_dataset_test = [item for item in detect_rewrite_dataset if item['REQID'] in test_ids]
detect_rewrite_dataset_train = [item for item in detect_rewrite_dataset if item['REQID'] not in test_ids]

generate_dataset_test = [item for item in generate_dataset if item['REQID'] in test_ids]
generate_dataset_train = [item for item in generate_dataset if item['REQID'] not in test_ids]

# 查看结果
print(f"Number of samples in the detect_rewrite_dataset_test: {len(detect_rewrite_dataset_test)}")
print(f"Number of samples in the detect_rewrite_dataset_train: {len(detect_rewrite_dataset_train)}")
print(f"Number of samples in the generate_dataset_test: {len(generate_dataset_test)}")
print(f"Number of samples in the generate_dataset_train: {len(generate_dataset_train)}")

In [ ]:
print(detect_rewrite_dataset_test)

In [ ]:
# {"text": "<s>[INST] What does OPL stand for in the OPL programming language? [/INST] OPL is short for Open Programming Language </s>"}
def to_llama(jsonl_file):
    _to_LLAMA = []
    for j_line in jsonl_file:
        if len(j_line['label']) > 0:
            completion = j_line['label'][0]
        prompt = j_line['text']
        j_line_LLAMA = {'text': f'<s>[INST] {prompt} [/INST] {completion}'}
        _to_LLAMA.append(j_line_LLAMA)
    return _to_LLAMA


def to_zepyra_detect_rewrite(jsonl_file):
    _to_zepyra = []
    for j_line in jsonl_file:
        if len(j_line['label']) > 0:
            completion = j_line['label'][0]
        prompt = j_line['text']
        j_line_zepyra = {'text': f'<|system|>\n You are a professional requirements engineer who helps users detect Transformational Effects of the given requirement and rewrite to eliminate them. \n <|user|> \n {prompt} \n<|assistant|> \n {completion}'}
        _to_zepyra.append(j_line_zepyra)
    return _to_zepyra


def to_zepyra_generate(jsonl_file):
    _to_zepyra = []
    for j_line in jsonl_file:
        if len(j_line['label']) > 0:
            completion = j_line['label'][0]
        prompt = j_line['text']
        j_line_zepyra = {'text': f'<|system|>\n You are a professional requirements engineer who helps users generate requirements free of Transformational Effects. \n <|user|> \n {prompt} \n<|assistant|> \n {completion}'}
        _to_zepyra.append(j_line_zepyra)
    return _to_zepyra

# LLAMA 3
def to_llama3_detect_rewrite(jsonl_file):
    _to_llama3 = []
    for j_line in jsonl_file:
        if len(j_line['label']) > 0:
            completion = j_line['label'][0]
        prompt = j_line['text']
        j_line_llama3 = {'input_for_llama3': f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n You are a professional requirements engineer who helps users detect Transformational Effects of the given requirement and rewrite to eliminate them. \n <|eot_id|><|start_header_id|>user<|end_header_id|> \n {prompt} \n<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
                         'completion': f'{completion}'
                        }
        _to_llama3.append(j_line_llama3)
    return _to_llama3


def to_llama3_generate(jsonl_file):
    _to_llama3 = []
    for j_line in jsonl_file:
        if len(j_line['label']) > 0:
            completion = j_line['label'][0]
        prompt = j_line['text']
        j_line_llama3 = {'input_for_llama3': f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n You are a professional requirements engineer who helps users generate requirements free of Transformational Effects. \n <|eot_id|><|start_header_id|>user<|end_header_id|> \n {prompt} \n<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
                         'completion': f'{completion}'
                        }
        _to_llama3.append(j_line_llama3)
    return _to_llama3


# TODO
def to_Komodo(jsonl_file):
    _to_Komodo = []
    for j_line in jsonl_file:
        if len(j_line['label']) > 0:
            completion = j_line['label'][0]
        prompt = j_line['text']
        j_line_Komodo = {'text': f'### Instruction: {prompt} ### Assistant: {completion}'}
        _to_Komodo.append(j_line_Komodo)
    return _to_Komodo

In [ ]:
my_zepyra_detect_rewrite_dataset_test = datasets.Dataset.from_pandas(pd.DataFrame(to_zepyra_detect_rewrite(detect_rewrite_dataset_test)))
my_zepyra_detect_rewrite_dataset_train = datasets.Dataset.from_pandas(pd.DataFrame(to_zepyra_detect_rewrite(detect_rewrite_dataset_train)))

my_zepyra_generate_dataset_test = datasets.Dataset.from_pandas(pd.DataFrame(to_zepyra_generate(generate_dataset_test)))
my_zepyra_generate_dataset_train = datasets.Dataset.from_pandas(pd.DataFrame(to_zepyra_generate(generate_dataset_train)))

my_zepyra_detect_rewrite_dataset_test.save_to_disk('my_zepyra_detect_rewrite_dataset_test_17040204')
my_zepyra_detect_rewrite_dataset_train.save_to_disk('my_zepyra_detect_rewrite_dataset_train_17042024')
my_zepyra_generate_dataset_test.save_to_disk('my_zepyra_generate_dataset_test_17042024')
my_zepyra_generate_dataset_train.save_to_disk('my_zepyra_generate_dataset_train_17042024')

In [ ]:
# my_LLAMA_dataset = datasets.Dataset.from_pandas(pd.DataFrame(to_LLAMA(init_dataset)))
# my_LLAMA_dataset.save_to_disk('llama_instructions_13042024')



from datasets import load_dataset, concatenate_datasets

my_zepyra_combined_dataset_train = concatenate_datasets([my_zepyra_detect_rewrite_dataset_train, my_zepyra_generate_dataset_train])
my_zepyra_combined_dataset_test = concatenate_datasets([my_zepyra_detect_rewrite_dataset_test, my_zepyra_generate_dataset_test])
my_zepyra_combined_dataset_train.save_to_disk('my_zepyra_combined_dataset_train_17040204')
my_zepyra_combined_dataset_test.save_to_disk('my_zepyra_combined_dataset_test_17040204')


In [ ]:
# my_Komodo_dataset = datasets.Dataset.from_pandas(pd.DataFrame(to_Komodo(init_dataset)))
# my_Komodo_dataset.save_to_disk('Komodo_instructions_13042024')

print(my_zepyra_combined_dataset_train[100]['text'])

In [ ]:
# LLAMA
my_llama_detect_rewrite_dataset_test = datasets.Dataset.from_pandas(pd.DataFrame(to_llama(detect_rewrite_dataset_test)))
my_llama_detect_rewrite_dataset_train = datasets.Dataset.from_pandas(pd.DataFrame(to_llama(detect_rewrite_dataset_train)))

my_llama_generate_dataset_test = datasets.Dataset.from_pandas(pd.DataFrame(to_llama(generate_dataset_test)))
my_llama_generate_dataset_train = datasets.Dataset.from_pandas(pd.DataFrame(to_llama(generate_dataset_train)))

my_llama_detect_rewrite_dataset_test.save_to_disk('my_llama_detect_rewrite_dataset_test_25040204')
my_llama_detect_rewrite_dataset_train.save_to_disk('my_llama_detect_rewrite_dataset_train_25042024')
my_llama_generate_dataset_test.save_to_disk('my_llama_generate_dataset_test_25042024')
my_llama_generate_dataset_train.save_to_disk('my_llama_generate_dataset_train_25042024')

from datasets import load_dataset, concatenate_datasets

my_llama_combined_dataset_train = concatenate_datasets([my_llama_detect_rewrite_dataset_train, my_llama_generate_dataset_train])
my_llama_combined_dataset_test = concatenate_datasets([my_llama_detect_rewrite_dataset_test, my_llama_generate_dataset_test])
my_llama_combined_dataset_train.save_to_disk('my_llama_combined_dataset_train_25040204')
my_llama_combined_dataset_test.save_to_disk('my_llama_combined_dataset_test_25040204')

print(my_zepyra_combined_dataset_train[100]['text'])

In [ ]:
# LLAMA 3
my_llama3_detect_rewrite_dataset_test = datasets.Dataset.from_pandas(pd.DataFrame(to_llama3_detect_rewrite(detect_rewrite_dataset_test)))
my_llama3_detect_rewrite_dataset_train = datasets.Dataset.from_pandas(pd.DataFrame(to_llama3_detect_rewrite(detect_rewrite_dataset_train)))

my_llama3_generate_dataset_test = datasets.Dataset.from_pandas(pd.DataFrame(to_llama3_generate(generate_dataset_test)))
my_llama3_generate_dataset_train = datasets.Dataset.from_pandas(pd.DataFrame(to_llama3_generate(generate_dataset_train)))

my_llama3_detect_rewrite_dataset_test.save_to_disk('my_llama3_detect_rewrite_dataset_test_07052024')
my_llama3_detect_rewrite_dataset_train.save_to_disk('my_llama3_detect_rewrite_dataset_train_07052024')
my_llama3_generate_dataset_test.save_to_disk('my_llama3_generate_dataset_test_07052024')
my_llama3_generate_dataset_train.save_to_disk('my_llama3_generate_dataset_train_07052024')

from datasets import load_dataset, concatenate_datasets

my_llama3_combined_dataset_train = concatenate_datasets([my_llama3_detect_rewrite_dataset_train, my_llama3_generate_dataset_train])
my_llama3_combined_dataset_test = concatenate_datasets([my_llama3_detect_rewrite_dataset_test, my_llama3_generate_dataset_test])
my_llama3_combined_dataset_train.save_to_disk('my_llama3_combined_dataset_train_07052024')
my_llama3_combined_dataset_test.save_to_disk('my_llama3_combined_dataset_test_07052024')

print(my_llama3_combined_dataset_train[100]['input_for_llama3'])

In [ ]:
print(my_llama3_combined_dataset_train[10]['completion'])

In [ ]:
def combine_columns(example):
    example["text"] = example["input_for_llama3"] + example["completion"]
    return example

# 对数据集进行转换，将 "text" 列添加到数据集中
dataset_for_training = my_llama3_combined_dataset_train.map(combine_columns)
print(dataset_for_training)

In [ ]:
print(dataset_for_training['text'])

In [ ]:
dataset_for_training.save_to_disk('my_llama3_combined_dataset_for_training_07052024')